In [10]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
from matplotlib import pyplot as plt
import matplotlib
import fastText
import html
import string
import nltk
from multiprocessing import Process
import multiprocessing
import time
import sys
from importlib import reload
import redis
# from urllib import request
from urllib.parse import urlparse
import requests as req
# from lxml import html
import logging
# from scrapy.http import Request
import requests
import json
import requests
from scrapy.http import TextResponse
from bs4 import BeautifulSoup
import re
from collections import Counter
from sklearn.metrics import precision_recall_curve
from tabulate import tabulate
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from gensim.models import Word2Vec

import pandas as pd
import math
import mpld3
import pickle

mpld3.enable_notebook()
%matplotlib inline

In [82]:
import utils
utils = reload(utils)

In [3]:
df_comparisons = pd.read_csv('df_comparisons_unsup_lemma.csv', index_col=0)
df_news = pd.read_csv('df_news_unsup_lemma.csv', parse_dates=['Datetime', 'Date'])

In [4]:
fields = ['title', 'spot','text']

In [5]:
for field in fields:
    df_news[field+'_pp_str'].fillna('', inplace=True)
    df_news[field+'_pp'] = df_news[field+'_pp_str'].apply(lambda x: x.split())

In [11]:
ftmodel = fastText.load_model('../../data/model/downloads/haber-P1_S1_L1-100E.bin')
model_w2v = Word2Vec.load('../../data/model/downloads/reaction.all.1544173485.punct_True.stopwords_True.lemmatized_True.w2v_100V_5E_1547415130.bin')

In [41]:
Xs_ft = {field: np.concatenate(df_news[field+'_pp_str'].apply(lambda doc: ftmodel.get_sentence_vector(doc).reshape(1,-1)).values, axis=0) for field in fields}

In [42]:
for field in fields:
    print(field, Xs_ft[field].shape)

title (2049, 100)
spot (2049, 100)
text (2049, 100)


In [27]:
def get_w2v_vector(words_now):
    
    vecnow = np.zeros((model_w2v.wv.vector_size,))
    count = 0
    
    for word in words_now:
        if word in model_w2v.wv.vocab:
            vecnow += model_w2v.wv.word_vec(word)
            count += 1
            
    if count == 0:
        return vecnow
    else:
        return vecnow / count`

In [50]:
Xs_wv = {field: np.concatenate(df_news[field + '_pp'].apply(lambda x: get_w2v_vector(x).reshape(1,-1)).values, axis=0) for field in fields}

In [51]:
for field in fields:
    print(field, Xs_wv[field].shape)

title (2049, 100)
spot (2049, 100)
text (2049, 100)


In [56]:
field

'text'

In [61]:
def concat_feats(rownow):
    
    x1 = Xs_wv[field][int(rownow.ind0),:].reshape(1,-1)
    x2 = Xs_wv[field][int(rownow.ind1),:].reshape(1,-1)
    
    print(x1.shape, x2.shape)
    
    return np.concatenate((x1, x2), axis=1)

In [87]:
for field in fields:
    df_comparisons['feat_concat_wv_' + field] = df_comparisons.apply(lambda row:
                                                                     np.concatenate((Xs_wv[field][int(row.ind0),:].reshape(1,-1), Xs_wv[field][int(row.ind1),:].reshape(1,-1)), axis=1), axis=1)
                                                                     

                                                                     
# F_concat_wv = {field: np.concatenate(df_comparisons.apply(lambda row: np.concatenate((Xs_wv[field][int(row.ind0),:].reshape(1,-1), Xs_wv[field][int(row.ind1),:].reshape(1,-1)), axis=1), axis=1).values,
#                axis=0) for field in fields}

In [88]:
for field in fields:
    df_comparisons['feat_concat_ft_' + field] = df_comparisons.apply(lambda row:
                                                                     np.concatenate((Xs_ft[field][int(row.ind0),:].reshape(1,-1), Xs_ft[field][int(row.ind1),:].reshape(1,-1)), axis=1), axis=1)


In [89]:
n_val_splits = 10

dfs_val_comparisons = utils.split_val_parts(df_comparisons, n_val_splits)

0
(1672, 47)
label
0.0    1501
1.0     171
Name: label, dtype: int64

1
(1719, 47)
label
0.0    1535
1.0     184
Name: label, dtype: int64

2
(1664, 47)
label
0.0    1467
1.0     197
Name: label, dtype: int64

3
(1671, 47)
label
0.0    1485
1.0     186
Name: label, dtype: int64

4
(1743, 47)
label
0.0    1561
1.0     182
Name: label, dtype: int64

5
(1620, 47)
label
0.0    1444
1.0     176
Name: label, dtype: int64

6
(1636, 47)
label
0.0    1455
1.0     181
Name: label, dtype: int64

7
(1668, 47)
label
0.0    1463
1.0     205
Name: label, dtype: int64

8
(1758, 47)
label
0.0    1577
1.0     181
Name: label, dtype: int64

9
(1707, 47)
label
0.0    1512
1.0     195
Name: label, dtype: int64



In [104]:
def print_prec_recall(colnames, return_res=False, dfnow=None):
    
    if dfnow is None:
        dfnow = df_comparisons
        
    if not isinstance(colnames, list):
        colnames = [colnames]
    
    table = []
    res = {}
    for colname in colnames:
    
        predictions = dfnow[colname]
        labels = dfnow['label']

        acc = float((predictions == labels).sum() / len(predictions))

        true_positive = float(np.logical_and(predictions==1, labels==1).sum())
        true_negative = float(np.logical_and(predictions==0, labels==0).sum())
        false_positive = float(np.logical_and(predictions==1, labels==0).sum())
        false_negative = float(np.logical_and(predictions==0, labels==1).sum())

        if predictions.sum() == 0 or predictions.sum() == predictions.shape[0]:
            print("Warning! number of positive predictions: %i" % predictions.sum())
            precision = 0 
            recall = 0
            f1 = 0
        else:
            precision = float(true_positive / (true_positive + false_positive))
            recall = float(true_positive / (true_positive + false_negative))
            f1 = 2*(recall * precision) / (recall + precision)
    
        table.append([colname, acc, f1, precision, recall, true_positive, true_negative, false_positive, false_negative])
        
        res[colname] = dict(
            acc = acc,
            f1 = f1,
            precision = precision,
            recall = recall,
            true_positive = true_positive,
            true_negative = true_negative,
            false_positive = false_positive,
            false_negative = false_negative)

    print(tabulate(table, headers=['METHOD', 'ACC', 'F1', 'PREC', 'RECALL', 'TP', 'TN', 'FP', 'FN'], tablefmt="fancy_grid"))
    
    if return_res:
        return res

In [102]:
def cross_validate_concat(df_list, df_comparisons, classifier_func, colname, columns, n_split, labelcol='label'):
    
    models = []

    for i in range(n_split):
        split_inds = [ii for ii in range(n_split)]
        split_inds.remove(i)
        df_train_now = pd.concat([df_list[ii] for ii in split_inds], axis=0)
        df_test_now = df_list[i]

        Xtrain = np.concatenate(df_train_now[columns].values, axis=0)
        ytrain = df_train_now[labelcol].values

        Xtest = np.concatenate(df_test_now[columns].values, axis=0)
        # ytest = df_test_now[labelcol].values
        
#         print(Xtrain.shape, ytrain.shape)
        model = classifier_func()
        model = model.fit(Xtrain, ytrain)

        yhat = model.predict(Xtest)
        df_comparisons.loc[df_test_now.index, colname] = yhat

        models.append(model)

    return models, df_comparisons

__Fasttext Results__

In [103]:
field = 'text'
classifier_name = 'dt'
wv_model_name = 'ft'

out_colname = 'feat_concat_ft_'+field+'_'+classifier_name
in_colname = 'feat_concat_ft_'+field

classifier_func = lambda: DecisionTreeClassifier(max_depth=None, criterion='entropy', min_samples_leaf=10)
trees, df_comparisons = cross_validate_concat(dfs_val_comparisons, df_comparisons, classifier_func, out_colname, in_colname, n_val_splits)

In [105]:
print_prec_recall(out_colname)

╒════════════════════════╤══════════╤══════════╤══════════╤══════════╤══════╤═══════╤══════╤══════╕
│ METHOD                 │      ACC │       F1 │     PREC │   RECALL │   TP │    TN │   FP │   FN │
╞════════════════════════╪══════════╪══════════╪══════════╪══════════╪══════╪═══════╪══════╪══════╡
│ feat_concat_ft_text_dt │ 0.897022 │ 0.518847 │ 0.534857 │ 0.503767 │  936 │ 14186 │  814 │  922 │
╘════════════════════════╧══════════╧══════════╧══════════╧══════════╧══════╧═══════╧══════╧══════╛


In [106]:
field = 'text'
classifier_name = 'rf'
wv_model_name = 'ft'

out_colname = 'feat_concat_ft_'+field+'_'+classifier_name
in_colname = 'feat_concat_ft_'+field

classifier_func = lambda: RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None)
trees, df_comparisons = cross_validate_concat(dfs_val_comparisons, df_comparisons, classifier_func, out_colname, in_colname, n_val_splits)

In [107]:
print_prec_recall(out_colname)

╒════════════════════════╤══════════╤══════════╤══════════╤══════════╤══════╤═══════╤══════╤══════╕
│ METHOD                 │      ACC │       F1 │     PREC │   RECALL │   TP │    TN │   FP │   FN │
╞════════════════════════╪══════════╪══════════╪══════════╪══════════╪══════╪═══════╪══════╪══════╡
│ feat_concat_ft_text_rf │ 0.942164 │ 0.647633 │ 0.985699 │ 0.482239 │  896 │ 14987 │   13 │  962 │
╘════════════════════════╧══════════╧══════════╧══════════╧══════════╧══════╧═══════╧══════╧══════╛


In [108]:
field = 'text'
classifier_name = 'svm'
wv_model_name = 'ft'

out_colname = 'feat_concat_ft_'+field+'_'+classifier_name
in_colname = 'feat_concat_ft_'+field

classifier_func = lambda: SVC(C=1000000, kernel='rbf', gamma=1e-5)
svms, df_comparisons = cross_validate_concat(dfs_val_comparisons, df_comparisons, classifier_func, out_colname, in_colname, n_val_splits)

In [110]:
print_prec_recall(out_colname)

Warning! number of positive predictions: 0
╒═════════════════════════╤══════════╤══════╤════════╤══════════╤══════╤═══════╤══════╤══════╕
│ METHOD                  │      ACC │   F1 │   PREC │   RECALL │   TP │    TN │   FP │   FN │
╞═════════════════════════╪══════════╪══════╪════════╪══════════╪══════╪═══════╪══════╪══════╡
│ feat_concat_ft_text_svm │ 0.889785 │    0 │      0 │        0 │    0 │ 15000 │    0 │ 1858 │
╘═════════════════════════╧══════════╧══════╧════════╧══════════╧══════╧═══════╧══════╧══════╛


__Word2VEc results__

In [112]:
field = 'text'
classifier_name = 'rf'
wv_model_name = 'wv'

out_colname = 'feat_concat_ft_'+field+'_'+classifier_name
in_colname = 'feat_concat_ft_'+field

classifier_func = lambda: RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None)
trees, df_comparisons = cross_validate_concat(dfs_val_comparisons, df_comparisons, classifier_func, out_colname, in_colname, n_val_splits)

In [113]:
print_prec_recall(out_colname)

╒════════════════════════╤══════════╤══════════╤══════════╤══════════╤══════╤═══════╤══════╤══════╕
│ METHOD                 │      ACC │       F1 │     PREC │   RECALL │   TP │    TN │   FP │   FN │
╞════════════════════════╪══════════╪══════════╪══════════╪══════════╪══════╪═══════╪══════╪══════╡
│ feat_concat_ft_text_rf │ 0.941689 │ 0.645254 │ 0.979189 │ 0.481163 │  894 │ 14981 │   19 │  964 │
╘════════════════════════╧══════════╧══════════╧══════════╧══════════╧══════╧═══════╧══════╧══════╛


In [114]:
field = 'text'
classifier_name = 'svm'
wv_model_name = 'wv'

out_colname = 'feat_concat_ft_'+field+'_'+classifier_name
in_colname = 'feat_concat_ft_'+field

classifier_func = lambda: SVC(C=1000000, kernel='rbf', gamma=1e-5)
svms, df_comparisons = cross_validate_concat(dfs_val_comparisons, df_comparisons, classifier_func, out_colname, in_colname, n_val_splits)

In [115]:
print_prec_recall(out_colname)

Warning! number of positive predictions: 0
╒═════════════════════════╤══════════╤══════╤════════╤══════════╤══════╤═══════╤══════╤══════╕
│ METHOD                  │      ACC │   F1 │   PREC │   RECALL │   TP │    TN │   FP │   FN │
╞═════════════════════════╪══════════╪══════╪════════╪══════════╪══════╪═══════╪══════╪══════╡
│ feat_concat_ft_text_svm │ 0.889785 │    0 │      0 │        0 │    0 │ 15000 │    0 │ 1858 │
╘═════════════════════════╧══════════╧══════╧════════╧══════════╧══════╧═══════╧══════╧══════╛
